In [28]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [29]:
filename = "../../data/06/ABox.ttl"

In [30]:
text1 = '''CQ_6.1
What are the treatment options developed for each risk? For each option, what are its layer of enclosure, stage of control, capital cost, annual cost, and notes?
'''

query1 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT DISTINCT ?option ?note ?layer ?stage ?capital_cost ?annual_cost
WHERE {
  ?treatment hero:develops ?option .
  ?option hero:hasNote ?note ;
          hero:isClassifiedByLayer ?layer ;
          hero:isClassifiedByControlStage ?stage ;
          hero:hasCapitalCost ?capital_cost ;
          hero:hasMaintenanceCost ?annual_cost .
}
'''

In [31]:
text2 = '''CQ_6.2
Which are the treatment options existing in the "fitting" layer and at the BLOCK stage of control?
'''

query2 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT DISTINCT ?option
WHERE {
    ?treatment hero:develops ?option .
    ?option hero:isClassifiedByLayer hero:fitting ;
            hero:isClassifiedByControlStage hero:block .
}
'''

In [32]:
text3 = '''CQ_6.3
Which are the treatment options with a capital cost higher than 1000 and an annual cost lower than 100?
'''

query3 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT ?option ?capital_cost ?annual_cost
WHERE {
    ?treatment hero:develops ?option .
    ?option hero:hasCapitalCost ?capital_cost ;
            hero:hasMaintenanceCost ?annual_cost .
    FILTER(
      ?capital_cost > 1000 &&
      ?annual_cost < 100
    )
}
'''

In [33]:
text4 = '''CQ_6.4
Which are the treatment options existing in the "building" or "room" layer and at the AVOID or DETECT stage of control, with a capital cost higher than 50 and an annual cost lower than 50?
'''

query4 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT ?option ?note ?layer ?stage ?capital_cost ?annual_cost
WHERE {
    ?treatment hero:develops ?option .
    ?option hero:hasNote ?note ;
            hero:isClassifiedByLayer ?layer ;
            hero:isClassifiedByControlStage ?stage ;
            hero:hasCapitalCost ?capital_cost ;
            hero:hasMaintenanceCost ?annual_cost .
    FILTER(
      (
        ?layer = hero:building ||
        ?layer = hero:room
      ) &&
      (
        ?stage = hero:avoid ||
        ?stage = hero:detect
      ) &&
      (
        ?capital_cost > 50 &&
        ?annual_cost < 50
      )
    )
}
'''

In [34]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3),
           (text4, query4)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_6.1
What are the treatment options developed for each risk? For each option, what are its layer of enclosure, stage of control, capital cost, annual cost, and notes?

+-----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+----------------------------------------------+----------------+---------------+
| option                                                          | note                                                                                                                                                         | layer                                          | stage                                        |   capital_cost |   annual_cost |
|-----------------------------------------------------------------+---------------------------------------------------